<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [3]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [4]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJFINANCE', 'BERGEPAINT', 'BHARTIARTL',
       'BRITANNIA', 'CANBK', 'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR',
       'DIVISLAB', 'DMART', 'DRREDDY', 'EICHERMOT', 'FEDERALBNK',
       'GODREJCP', 'GODREJIND', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK',
       'ICICIPRULI', 'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTI', 'M&M', 'MARUTI',
       'MCDOWELL-N', 'MINDTREE', 'MOTILALOFS', 'MUTHOOTFIN', 'NATIONALUM',
       'NAUKRI', 'PIDILITIND', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN',
       'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL',
       'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO', 'ZEEL'],
      dtype=object)

In [5]:
mypf['InPortfolio'].value_counts()

0    44
1    17
Name: InPortfolio, dtype: int64

In [6]:
mypf[(mypf['Symbol'] + '.NS') == (stock_ids[1] + '.NS')]['Forecast'].values[0]

1044.0

In [7]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
import talib 
yfin.pdr_override()

In [8]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'EMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = web.DataReader(stock_symbol,  data_source='yahoo', start = start, end = end)['Close']
    stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close Price'].ewm(span = short_window, adjust = False).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close Price'].ewm(span = mid_window, adjust = False).mean(),0)     

    stock_df['RSI'] = talib.RSI(stock_df['Close Price'], timeperiod=14)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close Price'].ewm(span = long_window, adjust = False).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close Price'] - stock_df['200_EMA'])*100/stock_df['200_EMA'],2)
    stock_df['Max'] = round(max(stock_df['Close Price']),0)
    stock_df['Min'] = round(min(stock_df['Close Price']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close Price']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close Price'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Close Price'] = round(stock_df['Close Price'],0) 
    
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    print(stock_id)
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

ASIANPAINT.NS
[*********************100%***********************]  1 of 1 completed
AXISBANK.NS
[*********************100%***********************]  1 of 1 completed
BAJFINANCE.NS
[*********************100%***********************]  1 of 1 completed
BERGEPAINT.NS
[*********************100%***********************]  1 of 1 completed
BHARTIARTL.NS
[*********************100%***********************]  1 of 1 completed
BRITANNIA.NS
[*********************100%***********************]  1 of 1 completed
CANBK.NS
[*********************100%***********************]  1 of 1 completed
CIPLA.NS
[*********************100%***********************]  1 of 1 completed
COLPAL.NS
[*********************100%***********************]  1 of 1 completed
DABUR.NS
[*********************100%***********************]  1 of 1 completed
DEEPAKNTR.NS
[*********************100%***********************]  1 of 1 completed
DIVISLAB.NS
[*********************100%***********************]  1 of 1 completed
DMART.NS
[*******************

In [9]:
stock_prec_dev('^NSEI').tail(1)

[*********************100%***********************]  1 of 1 completed


,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%
Date,,,,,,,,,,,
2022-11-17,18408.0,18042.0,17708.0,68.802282,17273.0,^NSEI,6.57,18410.0,15294.0,99.93,0.07


In [10]:
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close Price'])*100/(df_prec_dev['Close Price']),2)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)

In [11]:
# buy or accumulate (based on std dev from 200 EMA)
df_prec_dev.sort_values(by = 'Dev%_200', ascending=True).head(10)

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
6,2022-11-17,3297.0,3467.0,3566.0,35.526427,3878.0,DIVISLAB.NS,-14.97,4938.0,3271.0,1.59,98.41,4065.0,23.29
32,2022-11-17,397.0,393.0,398.0,54.751489,465.0,WIPRO.NS,-14.56,722.0,375.0,6.43,93.57,433.0,9.07
0,2022-11-17,410.0,421.0,434.0,32.773717,474.0,GODREJIND.NS,-13.52,644.0,406.0,1.64,98.36,638.0,55.61
1,2022-11-17,685.0,694.0,716.0,40.255581,789.0,MOTILALOFS.NS,-13.12,980.0,682.0,1.16,98.84,930.0,35.77
15,2022-11-17,82.0,79.0,78.0,58.700390,93.0,IBREALEST.NS,-12.20,181.0,60.0,17.89,82.11,95.0,15.85
10,2022-11-17,75.0,74.0,74.0,55.238815,85.0,NATIONALUM.NS,-11.29,130.0,67.0,13.33,86.67,91.0,21.33
2,2022-11-17,481.0,500.0,518.0,33.399258,540.0,ICICIPRULI.NS,-10.94,643.0,447.0,17.32,82.68,647.0,34.51
30,2022-11-17,1055.0,1052.0,1052.0,51.443084,1180.0,TECHM.NS,-10.56,1806.0,965.0,10.74,89.26,1156.0,9.57
7,2022-11-17,3962.0,3991.0,3993.0,49.263591,4355.0,NAUKRI.NS,-9.02,6545.0,3375.0,18.52,81.48,4851.0,22.44
13,2022-11-17,556.0,583.0,592.0,34.564696,603.0,JUBLFOOD.NS,-7.80,786.0,464.0,28.56,71.44,651.0,17.09


In [12]:
# sell or downsize (based on std dev from 200 EMA)
df_prec_dev.sort_values(by = 'Dev%_200', ascending=False).head(15)

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
52,2022-11-17,308.0,290.0,266.0,72.496820,235.0,CANBK.NS,31.04,314.0,175.0,95.65,4.35,310.0,0.65
23,2022-11-17,134.0,133.0,127.0,56.280264,110.0,FEDERALBNK.NS,22.14,138.0,81.0,93.60,6.40,149.0,11.19
49,2022-11-17,345.0,347.0,338.0,48.994551,297.0,ITC.NS,16.06,361.0,208.0,89.35,10.65,350.0,1.45
12,2022-11-17,603.0,586.0,564.0,63.104006,522.0,SBIN.NS,15.46,616.0,435.0,92.65,7.35,709.0,17.58
44,2022-11-17,845.0,819.0,792.0,68.122255,741.0,BHARTIARTL.NS,14.02,845.0,641.0,99.95,0.05,879.0,4.02
9,2022-11-17,860.0,854.0,820.0,55.433703,755.0,AXISBANK.NS,13.94,915.0,626.0,81.06,18.94,1044.0,21.40
26,2022-11-17,1270.0,1296.0,1276.0,43.633190,1125.0,M&M.NS,12.92,1354.0,691.0,87.38,12.62,1404.0,10.55
14,2022-11-17,921.0,906.0,888.0,59.234966,818.0,ICICIBANK.NS,12.57,926.0,654.0,98.09,1.91,1075.0,16.72
57,2022-11-17,4121.0,3967.0,3852.0,67.688941,3665.0,BRITANNIA.NS,12.44,4175.0,3094.0,95.00,5.00,3935.0,-4.51
42,2022-11-17,1012.0,1007.0,971.0,57.204879,901.0,SUNPHARMA.NS,12.37,1052.0,737.0,87.44,12.56,1056.0,4.35


In [13]:
# portfolio buy/sell
mypf['Stock'] = mypf['Symbol'] + '.NS'
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-11-17,410.0,421.0,434.0,32.773717,474.0,GODREJIND.NS,-13.52,644.0,406.0,1.64,98.36,638.0,55.61
1,2022-11-17,685.0,694.0,716.0,40.255581,789.0,MOTILALOFS.NS,-13.12,980.0,682.0,1.16,98.84,930.0,35.77
2,2022-11-17,481.0,500.0,518.0,33.399258,540.0,ICICIPRULI.NS,-10.94,643.0,447.0,17.32,82.68,647.0,34.51
3,2022-11-17,533.0,535.0,540.0,48.008518,569.0,HDFCLIFE.NS,-6.27,711.0,504.0,14.18,85.82,683.0,28.14
4,2022-11-17,1257.0,1254.0,1251.0,51.240051,1205.0,SBILIFE.NS,4.30,1332.0,1025.0,75.50,24.50,1577.0,25.46
5,2022-11-17,1759.0,1776.0,1808.0,45.303810,1884.0,INDIGO.NS,-6.65,2332.0,1549.0,26.79,73.21,2186.0,24.28
6,2022-11-17,3297.0,3467.0,3566.0,35.526427,3878.0,DIVISLAB.NS,-14.97,4938.0,3271.0,1.59,98.41,4065.0,23.29
7,2022-11-17,3962.0,3991.0,3993.0,49.263591,4355.0,NAUKRI.NS,-9.02,6545.0,3375.0,18.52,81.48,4851.0,22.44
10,2022-11-17,75.0,74.0,74.0,55.238815,85.0,NATIONALUM.NS,-11.29,130.0,67.0,13.33,86.67,91.0,21.33
15,2022-11-17,82.0,79.0,78.0,58.700390,93.0,IBREALEST.NS,-12.20,181.0,60.0,17.89,82.11,95.0,15.85


In [14]:
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%']*100/df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%'].sum()

0     16.546656
1     10.643299
2     10.268388
3      8.373006
4      7.575577
5      7.224470
6      6.929898
7      6.676982
10     6.346703
15     4.716139
21     3.421804
24     3.284932
25     3.151035
29     2.886218
43     1.202095
48     0.487979
51     0.264818
Name: Upside%, dtype: float64

In [15]:
# prospects buy
df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
8,2022-11-17,1142.0,1151.0,1140.0,46.460628,1040.0,INDUSINDBK.NS,9.77,1264.0,778.0,74.83,25.17,1394.0,22.07
9,2022-11-17,860.0,854.0,820.0,55.433703,755.0,AXISBANK.NS,13.94,915.0,626.0,81.06,18.94,1044.0,21.40
11,2022-11-17,254.0,262.0,262.0,38.594785,264.0,ZEEL.NS,-3.98,371.0,209.0,27.47,72.53,302.0,18.90
12,2022-11-17,603.0,586.0,564.0,63.104006,522.0,SBIN.NS,15.46,616.0,435.0,92.65,7.35,709.0,17.58
13,2022-11-17,556.0,583.0,592.0,34.564696,603.0,JUBLFOOD.NS,-7.80,786.0,464.0,28.56,71.44,651.0,17.09
14,2022-11-17,921.0,906.0,888.0,59.234966,818.0,ICICIBANK.NS,12.57,926.0,654.0,98.09,1.91,1075.0,16.72
16,2022-11-17,427.0,421.0,422.0,54.206461,443.0,TATAMOTORS.NS,-3.57,530.0,372.0,34.94,65.06,493.0,15.46
17,2022-11-17,6935.0,7099.0,7125.0,40.414933,6933.0,BAJFINANCE.NS,0.03,7853.0,5284.0,64.26,35.74,7978.0,15.04
18,2022-11-17,2466.0,2521.0,2550.0,38.389590,2451.0,HINDUNILVR.NS,0.61,2702.0,1944.0,68.86,31.14,2828.0,14.68
19,2022-11-17,839.0,837.0,850.0,49.124530,850.0,GODREJCP.NS,-1.31,968.0,665.0,57.38,42.62,958.0,14.18


In [16]:
# CCI buy
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
6,2022-11-17,3297.0,3467.0,3566.0,35.526427,3878.0,DIVISLAB.NS,-14.97,4938.0,3271.0,1.59,98.41,4065.0,23.29
13,2022-11-17,556.0,583.0,592.0,34.564696,603.0,JUBLFOOD.NS,-7.80,786.0,464.0,28.56,71.44,651.0,17.09
18,2022-11-17,2466.0,2521.0,2550.0,38.389590,2451.0,HINDUNILVR.NS,0.61,2702.0,1944.0,68.86,31.14,2828.0,14.68
25,2022-11-17,1596.0,1538.0,1507.0,65.850163,1565.0,INFY.NS,1.96,1940.0,1365.0,40.10,59.90,1765.0,10.59
30,2022-11-17,1055.0,1052.0,1052.0,51.443084,1180.0,TECHM.NS,-10.56,1806.0,965.0,10.74,89.26,1156.0,9.57
32,2022-11-17,397.0,393.0,398.0,54.751489,465.0,WIPRO.NS,-14.56,722.0,375.0,6.43,93.57,433.0,9.07
33,2022-11-17,3086.0,3121.0,3191.0,42.154506,3172.0,ASIANPAINT.NS,-2.71,3576.0,2580.0,50.80,49.20,3361.0,8.91
35,2022-11-17,1223.0,1231.0,1254.0,45.716950,1262.0,HAVELLS.NS,-3.12,1418.0,1070.0,43.85,56.15,1327.0,8.50
37,2022-11-17,610.0,603.0,614.0,52.262848,652.0,BERGEPAINT.NS,-6.44,792.0,561.0,21.21,78.79,651.0,6.72
41,2022-11-17,2590.0,2670.0,2627.0,39.825252,2478.0,TITAN.NS,4.50,2770.0,1936.0,78.37,21.63,2715.0,4.83
